In [1]:
!pip install kaggle_environments==0.1.6
from kaggle_environments import make, evaluate


env = make("connectx", debug=True)

print(list(env.agents))

['random', 'negamax']


In [2]:
# Random agent
def random_agent(observation, configuration):
    from random import choice
    return choice([c for c in range(configuration.columns) if observation.board[c] == 0])

In [3]:
!pip install --upgrade h5py

In [9]:
# Create the DQN

import tensorflow as tf
from tensorflow import keras


action_space = 7


In [13]:
m_input = tf.keras.layers.Input(shape = (6,7,))
flatten = tf.keras.layers.Flatten()(input)
hidden_1 = tf.keras.layers.Dense(50, activation = 'relu')(flatten)
hidden_2 = tf.keras.layers.Dense(50, activation = 'relu')(hidden_1)
hidden_3 = tf.keras.layers.Dense(50, activation = 'relu')(hidden_2)
hidden_4 = tf.keras.layers.Dense(50, activation = 'relu')(hidden_3)
output = tf.keras.layers.Dense(action_space, activation = "linear")(hidden_4) 
model = tf.keras.models.Model(inputs = [input], outputs = [output])

TypeError: Inputs to a layer should be tensors. Got: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ffc9b652340>>

In [16]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(20, activation=tf.nn.relu, input_shape=(6,7)),  
  tf.keras.layers.Dense(20, activation=tf.nn.relu),
  tf.keras.layers.Dense(action_space)
])

In [24]:
import random
import numpy as np
def epsilonDecision(epsilon):
  action_decision = random.choices(['model','random'], weights = [1 - epsilon, epsilon])[0]
  return action_decision


In [20]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 6, 20)             160       
                                                                 
 dense_13 (Dense)            (None, 6, 20)             420       
                                                                 
 dense_14 (Dense)            (None, 6, 7)              147       
                                                                 
Total params: 727
Trainable params: 727
Non-trainable params: 0
_________________________________________________________________


In [25]:
def chooseAction(model, observation, epsilon):
    action_decision = epsilonDecision(epsilon)
    observation = np.array([observation])
    preds = model.predict(observation)
    weights = tf.nn.softmax(preds).numpy()[0]
    
    if action_decision == 'model':
        action = np.argmax(weights)
    if action_decision == 'random':
        action = random.randint(0,6)
        
    return int(action), weights

In [29]:
class Experience:
    def __init__(self):
        self.clear() 
        
    def clear(self):
        self.observations = []
        self.actions = []
        self.rewards = []
        
    def store_experience(self, new_obs, new_act, new_reward):
        self.observations.append(new_obs)
        self.actions.append(new_act)
        self.rewards.append(new_reward)

In [30]:
def checkValid(obs, action):
    valid_actions = set([0,1,2,3,4,5,6])
    
    if obs[0,action] != 0:
        valid_actions = valid_actions - set([action])
        action = random.choice(list(valid_actions))
    return action

In [31]:
def getReward(winner, state):

    if not state:
        reward = 0
    if state: 
        if winner == 1:
            reward = 50
        if winner == -1:
            reward = -50
        if winner == 0:
            reward = -50
            
    return reward